****STDL PROJECT NOTEBOOK****

**Phase 1: predict gene expression levels of one specific gene over given input biopsy images**

Assign GPU device

In [1]:
# the next 2 lines are to allow debugging with CUDA !
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  
print(f'cuda debugging allowed')

cuda debugging allowed


In [2]:
######
import torch
print(f'cuda device count: {torch.cuda.device_count()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(f'device name: {torch.cuda.get_device_name(0)}')
    print(f'torch.cuda.device(0): {torch.cuda.device(0)}')
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
# NOTE: important !!!!!!
# clearing out the cache before beginning
torch.cuda.empty_cache()

cuda device count: 1
Using device: cuda
device name: GeForce RTX 2080 Ti
torch.cuda.device(0): <torch.cuda.device object at 0x7fe8b832ac10>
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB



Import project files

In [3]:
# create code to reimport module if i change it
%load_ext autoreload
%autoreload 2

# note: path to project is: /home/roy.rubin/STDLproject/
import loadAndPreProcess
import deepNetworkArchitechture

perform pre processing actions

In [4]:
print("\nHi! welcome to the program :)\n")


Hi! welcome to the program :)



In [5]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
imageFolder = loadAndPreProcess.load_dataset_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [6]:
path_to_images_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData/images"
augmentedImageFolder = loadAndPreProcess.load_augmented_imageFolder_DS_from_images_folder(path_to_images_dir)


----- entered function load_dataset_from_pictures_folder -----

----- finished function load_dataset_from_pictures_folder -----



In [7]:
path_to_mtx_tsv_files_dir = "/home/roy.rubin/STDLproject/spatialGeneExpressionData"
matrix_dataframe, features_dataframe, barcodes_datafame = loadAndPreProcess.load_dataframes_from_mtx_and_tsv_new(path_to_mtx_tsv_files_dir)


----- entered function load_dataframes_from_mtx_and_tsv -----
started reading features.tsv
V  finished reading features.tsv
started reading barcodes.tsv
V  finished reading barcodes.tsv
started reading matrix.mtx. this might take some time ...
V  finished reading matrix.mtx

----- finished function load_dataframes_from_mtx_and_tsv -----



remove empty genes (i think there are only 3 of these ... need to verify)

In [8]:
%autoreload 2
matrix_dataframe, mapping_between_old_and_new_indices = loadAndPreProcess.cut_empty_genes(matrix_dataframe)  #TODO: uncomment later

cutting all genes (rows) that contain only zeros ...


In [9]:
# uncomment if wanted
# print(f'\note: this is the mapping_between_old_and_new_indices: \n{mapping_between_old_and_new_indices}')

In [10]:
gene_name = 'MKI67'
custom_DS_SingleValuePerImg = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=imageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)
custom_DS_SingleValuePerImg_augmented = loadAndPreProcess.STDL_Dataset_SingleValuePerImg(imageFolder=augmentedImageFolder, 
                                                               matrix_dataframe=matrix_dataframe, 
                                                               features_dataframe=features_dataframe, 
                                                               barcodes_datafame=barcodes_datafame, 
                                                               chosen_gene_name=gene_name)


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----


----- entering __init__ phase of  STDL_Dataset_SingleValuePerImg -----

----- finished __init__ phase of  STDL_Dataset_SingleValuePerImg -----



In [11]:
k = 10
custom_DS_KGenesWithHighestVariance = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_KGenesWithHighestVariance_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_KGenesWithHighestVariance -----
calculate variance of all columns from  matrix_dataframe - and choosing K genes with higest variance ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [12]:
k = 10
custom_DS_LatentTensor_NMF = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)
custom_DS_LatentTensor_NMF_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_NMF(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_NMF -----
performing NMF decomposition on main matrix dataframe ...

----- finished __init__ phase of  STDL_Dataset_LatentTensor -----



In [13]:
%autoreload 2

k = 10
custom_DS_LatentTensor_AE = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=imageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)
custom_DS_LatentTensor_AE_augmented = loadAndPreProcess.STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder(imageFolder=augmentedImageFolder, 
                                                                           matrix_dataframe=matrix_dataframe, 
                                                                           features_dataframe=features_dataframe, 
                                                                           barcodes_datafame=barcodes_datafame, 
                                                                           num_of_dims_k=k,
                                                                           device=device)


----- entering __init__ phase of  STDL_Dataset_KValuesPerImg_LatentTensor_AutoEncoder -----
initializing the autoencoder (this might take a while) ...

----- entered function return_trained_AE_net -----
--delete-- verify inside STDL_Dataset_matrix_df_for_AE_init __init__: self.data.shape: (23073, 4992)
note - number of (hidden) linear layers is supposed to be 3

entered __init__ of AutoencoderNet
****** begin training ******

iteration 1 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.003874091664329171 max loss 777.17919921875
               average loss 10.52483733889542

iteration 2 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.0036157523281872272 max loss 2788.699951171875
               average loss 10.449787451121258

iteration 3 of 3 epochs
batch 2000 of 2000 batches
finished inner loop.

in this epoch: min loss 0.003266484010964632 max loss 3020.447021484375
               average loss 11.027725053


display a few sample images

In [14]:
# TODO: code to display some sample images
# 

check sizes of a pair (x,y) from all different datasets

In [15]:
print(f'custom_DS_SingleValuePerImg (single gene):      x shape {custom_DS_SingleValuePerImg[0][0].shape}, y is an int')
print(f'custom_DS_SingleValuePerImg_augmented:          x shape {custom_DS_SingleValuePerImg_augmented[0][0].shape}, y is an int')
print("---------")
print(f'custom_DS_KGenesWithHighestVariance:            x shape {custom_DS_KGenesWithHighestVariance[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance[0][1].shape}')
print(f'custom_DS_KGenesWithHighestVariance_augmented:  x shape {custom_DS_KGenesWithHighestVariance_augmented[0][0].shape}, y shape {custom_DS_KGenesWithHighestVariance_augmented[0][1].shape}')
print("---------")
print(f'custom_DS_LatentTensor_NMF:                     x shape {custom_DS_LatentTensor_NMF[0][0].shape}, y shape {custom_DS_LatentTensor_NMF[0][1].shape}')
print(f'custom_DS_LatentTensor_NMF_augmented:           x shape {custom_DS_LatentTensor_NMF_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_NMF_augmented[0][1].shape}')
print("---------")
print(f'custom_DS_LatentTensor_AE:                      x shape {custom_DS_LatentTensor_AE[0][0].shape}, y shape {custom_DS_LatentTensor_AE[0][1].shape}')
print(f'custom_DS_LatentTensor_AE_augmented:            x shape {custom_DS_LatentTensor_AE_augmented[0][0].shape}, y shape {custom_DS_LatentTensor_AE_augmented[0][1].shape}')


custom_DS_SingleValuePerImg (single gene):      x shape torch.Size([3, 176, 176]), y is an int
custom_DS_SingleValuePerImg_augmented:          x shape torch.Size([3, 176, 176]), y is an int
---------
custom_DS_KGenesWithHighestVariance:            x shape torch.Size([3, 176, 176]), y shape (10,)
custom_DS_KGenesWithHighestVariance_augmented:  x shape torch.Size([3, 176, 176]), y shape (10,)
---------
custom_DS_LatentTensor_NMF:                     x shape torch.Size([3, 176, 176]), y shape (10,)
custom_DS_LatentTensor_NMF_augmented:           x shape torch.Size([3, 176, 176]), y shape (10,)
---------
custom_DS_LatentTensor_AE:                      x shape torch.Size([3, 176, 176]), y shape torch.Size([10])
custom_DS_LatentTensor_AE_augmented:            x shape torch.Size([3, 176, 176]), y shape torch.Size([10])


In [16]:
%autoreload 2
import executionModule

executionModule.runTest1_singleGenePrediction(custom_DS_SingleValuePerImg, device)
executionModule.runTest1_singleGenePrediction(custom_DS_SingleValuePerImg_augmented, device)
# executionModule.runTest2_allGenePrediction_dimReduction_KHighestVariances(stdl_ds2, device)
# executionModule.runTest3_allGenePrediction_dimReduction_NMF(stdl_ds2, device)
# executionModule.runTest4_allGenePrediction_dimReduction_AutoEncoder(stdl_ds4, device)


----- entered function runTest1 -----
verify size of ds_train 3050
verify size of ds_test 763
verify size of dl_train 122
verify size of dl_test 31
verify im_size torch.Size([3, 176, 176])
verify size of dl_test 31
verify batch_size is 25 
A single image's shape will be like x0.shape : torch.Size([3, 176, 176])
A single image's shape will be like x0.shape - after unsqueeze : torch.Size([1, 3, 176, 176])
verify in_size torch.Size([3, 176, 176])
note - number of convolutions is supposed to be 1
note - number of (hidden) linear layers is supposed to be 1
****** begin training ******

iteration 1 of 10 epochs
batch 30 of 30 batches
finished inner loop.

in this epoch: min loss 0.0010705800959840417 max loss 769.7974853515625
               average loss 27.75787791266727
               number of correct predictions: 473 / 750

iteration 2 of 10 epochs
batch 30 of 30 batches
finished inner loop.

in this epoch: min loss 0.001071264734491706 max loss 0.043691374361515045
               avera